In [1]:
import os, re, sys
import pandas as pd
from collections import Counter 

#!pip install fuzzywuzzy 
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#!pip install python-Levenshtein
import Levenshtein 

# Assign CUSIP to company name from job ads

Phai Phongthiengtham
- March 19, 2019

This notebook demonstrates how I match company name from job ads to Committee on Uniform Security Identification Procedures (CUSIP), in order to merge with Compustat dataset.

**Due to copyright restrictions, I am not authorized to publish either Compustat, job posting data, nor the full results.**

-----------

### Define string cleaning functions
First, we need to perform some cleaning on company names. There are two steps.

1. initial_clean( ): This function removes unneccessary words and substitutes abbrevations.
2. simplify_name( ): This function further simplifies names from initial_clean( ).  

This process has been manually checked to confirm that all Compustat company name is uniquely preserved, i,e., no two compustat company with the same simplified name. This is important as many compustat companies have similar names. For example, 

- BROOKFIELD BUSINESS PRTNR LP (CUSIP = G16234109), and
- BROOKFIELD PROPERTY PRTRS LP (CUSIP = G16249107).

If I simplified too much, then the these two companies would have the same simplified name.

In [2]:
def initial_clean(input_string):
    
    substitute = input_string.lower()
    substitute = re.sub('\\b' + '\s?- old' + '\\b' , ' ', substitute)
    substitute = re.sub('\\b' + '\s?- consol' + '\\b' , ' ' , substitute)
    substitute = re.sub('\\b' + '\s?- consolidated' + '\\b' , ' ', substitute)
    substitute = re.sub('\\b' + '\s?- cl' + '\\b' , ' ' , substitute)
    substitute = re.sub('\\b' + '\s?- redh' + '\\b' , ' ' , substitute)
    substitute = re.sub('\\b' + '\s?- corp' + '\\b' , ' corp ' , substitute)
    substitute = re.sub('\\b' + '\s?- adr' + '\\b' , ' ' , substitute)
    substitute = re.sub('\\b' + '\s?- ads' + '\\b' , ' ' , substitute)
    substitute = re.sub('\\b' + '\s?- inc' + '\\b' , ' inc ' , substitute)
    substitute = re.sub('\\b' + '\s?- lp' + '\\b' , ' lp ' , substitute)
    substitute = re.sub('\\b' + '\s?- pro' + '\\b' , ' pro ' , substitute)
    substitute = re.sub('\\b' + 'wal mart' + '\\b' , ' walmart ' , substitute)
    substitute = re.sub('\\b' + 'wal-mart' + '\\b' , ' walmart ' , substitute)
    substitute = re.sub('\\b' + "toys 'r' us" + '\\b' , ' toys r us  ' , substitute)
    substitute = re.sub('\\b' + 'and' + '\\b' , ' & ' , substitute)
    substitute = re.sub('\\b' + 'the' + '\\b' , ' ' , substitute)

    substitute = re.sub('\\b' + 'holdings' + '\\b' , ' hldgs ' , substitute)
    substitute = re.sub('\\b' + 'holding' + '\\b' , ' hldgs ' , substitute)
    substitute = re.sub('\\b' + 'limited' + '\\b' , ' ltd ' , substitute)
    substitute = re.sub('\\b' + 'corporations' + '\\b' , ' corp ' , substitute)
    substitute = re.sub('\\b' + 'corporation' + '\\b' , ' corp ' , substitute)
    substitute = re.sub('\\b' + 'international' + '\\b' , ' intl ' , substitute)
    substitute = re.sub('\\b' + 'incorporated' + '\\b' , ' inc ' , substitute)
    substitute = re.sub('\\b' + 'company' + '\\b' , ' co ' , substitute)
    substitute = re.sub('\\b' + 'entertainment' + '\\b' , ' entmt ' , substitute)
    substitute = re.sub('\\b' + 'products' + '\\b' , ' prods ' , substitute)
    substitute = re.sub('\\b' + 'opportunities' + '\\b' , ' opp ' , substitute)
    substitute = re.sub('\\b' + 'management' + '\\b' , ' mgmt ' , substitute)
    substitute = re.sub('\\b' + 'managements' + '\\b' , ' mgmt ' , substitute)
    substitute = re.sub('\\b' + 'income' + '\\b' , ' incm ' , substitute)
    substitute = re.sub('\\b' + 'technologies' + '\\b' , ' tech ' , substitute)
    substitute = re.sub('\\b' + 'national' + '\\b' , ' natl ' , substitute)
    substitute = re.sub('\\b' + 'u s' + '\\b' , ' us ' , substitute)
    substitute = re.sub('\\b' + 'bank' + '\\b' , ' bk ' , substitute)
    
    substitute = re.sub('\\b' + 'manufacturing' + '\\b' , ' mfg ' , substitute)
    substitute = re.sub('\\b' + 'laboratories' + '\\b' , ' labs ' , substitute)
    substitute = re.sub('\\b' + 'laboratory' + '\\b' , ' labs ' , substitute)
    substitute = re.sub('\\b' + 'lab' + '\\b' , ' labs ' , substitute)
    substitute = re.sub('\\b' + 'group' + '\\b' , ' grp ' , substitute)
    substitute = re.sub('\\b' + 'groups' + '\\b' , ' grp ' , substitute)
    substitute = re.sub('\\b' + 'partners' + '\\b' , ' prtnrs ' , substitute)
    substitute = re.sub('\\b' + '1-5' + '\\b' , ' 1to5 ' , substitute)

    substitute = re.sub('\\b' + 'angiotech pharmaceuticals us inc' + '\\b' , ' angiotech pharmaceuticals inc ' , substitute)
    substitute = re.sub('\\b' + 'farmers national banc corp oh' + '\\b' , ' farmers national banc corp ' , substitute)
    substitute = re.sub('\\b' + 'lyondellbasell industries nv' + '\\b' , ' lyondellbasell industries inc ' , substitute)
    substitute = re.sub('\\b' + 'newtek business services inc' + '\\b' , ' newtek business services corp ' , substitute)

    substitute = re.sub('\\b' + 'healthcare service grp' + '\\b' , ' healthcare services grp ' , substitute)
    substitute = re.sub('\\b' + 'mitchells & butler plc' + '\\b' , ' mitchells & butlers plc ' , substitute)
    substitute = re.sub('\\b' + 'j g wentworth co' + '\\b' , ' jg wentworth co ' , substitute)
    substitute = re.sub('\\b' + 'l s starrett co' + '\\b' , ' ls starrett co ' , substitute)
    substitute = re.sub('\\b' + 'jcpenney co inc' + '\\b' , ' jc penney co inc ' , substitute)
    substitute = re.sub('\\b' + 'blackstone grp l p' + '\\b' , ' blackstone grp lp ' , substitute)
    substitute = re.sub('\\b' + 'service master co' + '\\b' , ' servicemaster co ' , substitute)
    substitute = re.sub('\\b' + 'j m smucker co' + '\\b' , ' jm smucker co ' , substitute)
    substitute = re.sub('\\b' + 'towne bank' + '\\b' , ' townebank ' , substitute)
    substitute = re.sub('\\b' + 'gerdau s a' + '\\b' , ' gerdau sa ' , substitute)
    substitute = re.sub('\\b' + 'southern california edison co' + '\\b' , ' southern california edison ' , substitute)

    split_space = [w for w in re.split(' ',substitute) if not w=='']
    final_tokens = list()
    
    for token in split_space:
        
        clean_token = token
        
        clean_token = clean_token.replace('-old',' ')
        clean_token = clean_token.replace('(the)',' ')
        clean_token = clean_token.replace('(',' ')
        clean_token = clean_token.replace(')',' ')
        clean_token = clean_token.replace("'s",'s')
        clean_token = clean_token.replace("s'",'s')
        clean_token = clean_token.replace('|',' ')
        clean_token = clean_token.replace(',',' ')
        clean_token = clean_token.replace('-',' ')
        
        clean_token = clean_token.replace('-consol',' ')
        clean_token = clean_token.replace('-consolidated',' ')
        clean_token = clean_token.replace('-cl',' ')
        clean_token = clean_token.replace('-redh',' ')
        clean_token = clean_token.replace('-spn',' ')
        clean_token = clean_token.replace('corp-',' corp ')
        clean_token = clean_token.replace('-corp',' corp ')
        clean_token = clean_token.replace('-adr',' ')
        clean_token = clean_token.replace('-ads',' ')
        clean_token = clean_token.replace('inc-',' inc ')
        clean_token = clean_token.replace('-inc',' inc ')
        clean_token = clean_token.replace('-lp',' lp ') #limited partnership
        clean_token = clean_token.replace('us.',' us ')
        clean_token = clean_token.replace('u.s.',' us ')
        clean_token = clean_token.replace('/',' ')
                
        if re.findall(re.escape('.'),token): #remove period but NOT .com
            if not re.findall(re.escape('.')+'com', token):
                clean_token = clean_token.replace('.','')
        
        final_tokens.append(clean_token)

    clean_name = ' '.join(final_tokens)
    clean_name = ' '.join([w for w in re.split(' ',clean_name) if not w==''])

    return clean_name

#.................................................................................................#

remove_word = ['inc','plc','lp', 'llc','the','corp','co']
exception = ['alcoa','aramark','carnival','bancorp','cinemedia','spire','tivo','allergan']

def simplify_name(input_string):

    simplify_name = input_string.lower()
    simplify_name = re.sub('[^a-z0-9 ]','',simplify_name)
    
    # remove extra white spaces and sort tokens by alphabetical (allows for word swap)
    selected_token = sorted([w for w in re.split(' ',simplify_name) if not w=='']) 
    output_string = ' '.join(selected_token)
    
    # remove words in "remove_word", skip if contains any company in "exception"
    # NOTE: these exceptions are neccessary,
    # otherwise, simplify_name() will produce duplicate results.  
    # For example, "bancorp" appears in many companies in compustat 

    if not bool(set(exception).intersection(selected_token)):
        selected_token = [w for w in selected_token if not w in remove_word]
        output_string = ' '.join(selected_token)
    
    return output_string

### Import a list of company names from Compustat

- "conm": Company name
- "conml": Company legal name

In [3]:
input_filename =  'compustat_firm_name_with_cusip.txt'
df_cp = pd.read_csv(input_filename, sep = '\t')
df_cp.head()

,conm,conml,cusip
0,A&W REVENUE ROYALTIES INC FD,A&W Revenue Royalties Income Fund,000255109
1,AAC HOLDINGS INC,AAC Holdings Inc,000307108
2,AAON INC,AAON Inc,000360206
3,AAR CORP,AAR Corp,000361105
4,ABB LTD,ABB Ltd,000375204


- Perform cleaning steps

In [4]:
df_cp['clean_conm'] = df_cp['conm'].apply( lambda x: initial_clean(x) )
df_cp['clean_conml'] = df_cp['conml'].apply( lambda x: initial_clean(x) )

df_cp['simplified_conm'] = df_cp['clean_conm'].apply( lambda x: simplify_name(x) )
df_cp['simplified_conml'] = df_cp['clean_conml'].apply( lambda x: simplify_name(x) )

df_cp.head()

,conm,conml,cusip,clean_conm,clean_conml,simplified_conm,simplified_conml
0,A&W REVENUE ROYALTIES INC FD,A&W Revenue Royalties Income Fund,000255109,a&w revenue royalties inc fd,a&w revenue royalties incm fund,aw fd revenue royalties,aw fund incm revenue royalties
1,AAC HOLDINGS INC,AAC Holdings Inc,000307108,aac hldgs inc,aac hldgs inc,aac hldgs,aac hldgs
2,AAON INC,AAON Inc,000360206,aaon inc,aaon inc,aaon,aaon
3,AAR CORP,AAR Corp,000361105,aar corp,aar corp,aar,aar
4,ABB LTD,ABB Ltd,000375204,abb ltd,abb ltd,abb ltd,abb ltd


### Define a function that assigns CUSIP

1. For each company name from job ads, first perform step 1 cleaning process using initial_clean().
2. Then, check if there is a match with "clean_conm". If not, then move to "clean_conml"
3. If no match in step 2, further simplify company name using simplify_name(). 
4. Repeat step 2 using simplified name.

In [5]:
def assign_cusip(company_name, df_cp):
    
    clean_name = initial_clean(company_name)
    simplified_name = simplify_name(clean_name)
    
    if clean_name in df_cp['clean_conm'].values: 
        cusip = list(df_cp[df_cp['clean_conm'] == clean_name]['cusip'])[0]        
        print('match type 1: cusip = ' + cusip)
        
    elif clean_name in df_cp['clean_conml'].values: 
        cusip = list(df_cp[df_cp['clean_conml'] == clean_name]['cusip'])[0]
        print('match type 2: cusip = ' + cusip)
        
    elif simplified_name in df_cp['simplified_conm'].values: 
        cusip = list(df_cp[df_cp['simplified_conm'] == simplified_name]['cusip'])[0]
        print('match type 3: cusip = ' + cusip)
        
    elif simplified_name in df_cp['simplified_conml'].values: 
        cusip = list(df_cp[df_cp['simplified_conml'] == simplified_name]['cusip'])[0]
        print('match type 4: cusip = ' + cusip)
        
    else:
        cusip = '-9999' # missing value
        print('no match')
        
    # print out actual observation (only for illustration)
    if not cusip == '-9999':
        print('------------------------')
        print(df_cp[df_cp['cusip'] == cusip])
        
    return cusip 

- I present some matched examples from actual ads below:

In [6]:
company_name = 'ACCENTURE, INC.'
clean_name = initial_clean(company_name)
simplified_name = simplify_name(clean_name)

print('--- Original name ---')
print(company_name)
print('--- Initial cleaning ---')
print(clean_name)
print('--- Simplified name ---')
print(simplified_name)
print('--- Perform matching ---')
assign_cusip(company_name, df_cp)

--- Original name ---
ACCENTURE, INC.
--- Initial cleaning ---
accenture inc
--- Simplified name ---
accenture
--- Perform matching ---
match type 3: cusip = G1151C101
------------------------
               conm          conml      cusip     clean_conm    clean_conml  \
9893  ACCENTURE PLC  Accenture PLC  G1151C101  accenture plc  accenture plc   

     simplified_conm simplified_conml  
9893       accenture        accenture  


'G1151C101'

In [7]:
company_name = 'IBI Group Inc.'
clean_name = initial_clean(company_name)
simplified_name = simplify_name(clean_name)

print('--- Original name ---')
print(company_name)
print('--- Initial cleaning ---')
print(clean_name)
print('--- Simplified name ---')
print(simplified_name)
print('--- Perform matching ---')
assign_cusip(company_name, df_cp)

--- Original name ---
IBI Group Inc.
--- Initial cleaning ---
ibi grp inc
--- Simplified name ---
grp ibi
--- Perform matching ---
match type 1: cusip = 44925L103
------------------------
               conm          conml      cusip   clean_conm  clean_conml  \
4303  IBI GROUP INC  IBI Group Inc  44925L103  ibi grp inc  ibi grp inc   

     simplified_conm simplified_conml  
4303         grp ibi          grp ibi  


'44925L103'

In [8]:
company_name = 'The Habit Restaurants LLC'
clean_name = initial_clean(company_name)
simplified_name = simplify_name(clean_name)

print('--- Original name ---')
print(company_name)
print('--- Initial cleaning ---')
print(clean_name)
print('--- Simplified name ---')
print(simplified_name)
print('--- Perform matching ---')
assign_cusip(company_name, df_cp)

--- Original name ---
The Habit Restaurants LLC
--- Initial cleaning ---
habit restaurants llc
--- Simplified name ---
habit restaurants
--- Perform matching ---
match type 3: cusip = 40449J103
------------------------
                             conm                        conml      cusip  \
4028  HABIT RESTAURANTS INC (THE)  Habit Restaurants Inc (The)  40449J103   

                 clean_conm            clean_conml    simplified_conm  \
4028  habit restaurants inc  habit restaurants inc  habit restaurants   

       simplified_conml  
4028  habit restaurants  


'40449J103'

- NOTE: Repeat this step, if the ad also provides master company name.

### Fuzzy string matching

According to wikipedia, Fuzzy string matching is the technique of finding strings that match a pattern approximately (rather than exactly). For example:

In [9]:
print(fuzz.ratio("jc penny co inc", "jcpenny co inc"))

97


The higher score (maximum of 100) means the two strings are extremely similar. However, it is possible to have two similar strings refering to different companies. For example: 

In [10]:
print(fuzz.ratio("cca industries inc", "c&a industries inc"))

94


In [11]:
print(fuzz.ratio("wintrust financial corp", "intrust financial corp"))

98


The company "cca industries inc" is different from "c&a industries inc", and "wintrust financial corp" is different from "intrust financial corp". This technique can be used to suggest the "closest" company from Compustat. However, manual inspection is still required in order to confirm these results.   